# Towards variadic perfect forwarding

## Universal references

When a function template has a type parameter `T`, and an argument whose type is exactly `T &&`, the later is called a **universal reference**, because it is allowed to link either with a temporary value (right value), or with a non-temporary value (left value).

What for ?? This enables to write functions which can handle any mix of temporary values and/or non-temporary values (one might object that the same could be obtained with arguments by value, but this would imply an expensive systematic copy of all values). Let's see a fake simple function with only one argument:

In [2]:
#include<iostream>
template<typename T>
void foo( T && val )
 {
  if constexpr (std::is_rvalue_reference_v<decltype(val)>)
   { std::cout<<"&& "<<val<<std::endl ; }
  else
   { std::cout<<" & "<<val<<std::endl ; }
 }

If called with a temporary value (right value), then `T` is deduced of the same type as the value. On the contrary, if called with a non-temporary value, then `T` and the argument are considered as classic references:

In [3]:
float pi = 3.14159 ;
foo(42) ;
foo(pi) ;

&& 42
 & 3.14159


## std::forward

When a function receive a rvalue reference, and must transfer it to another function, we need to wrap it with `std::move` if we want to preserve it as a rvalue reference. In the case of a universal reference, this must be done only if the type is actually interpreted as a rvalue reference, and not if it is interpreted as a lvalue reference. That's the job of `std::forward`.

In [5]:
#include <iostream>
#include <utility>
template<typename T>
void do_stuff( T && t )
{
  std::cout<<"Preparing... " ;
  foo(std::forward<T>(t)) ;
}

In [6]:
float pi = 3.14159 ;
do_stuff(42) ;
do_stuff(pi) ;

Preparing... && 42
Preparing...  & 3.14159


We can qualify this as **perfect forwarding**, because the function `foo` receive an argument whose type is very exactly the same as the initial value given to `do_stuff`.

BUT, what if we want some kind of `do_stuff` which can receive any number of arguments, and perfectly forwards them some other function ?

## A taste of variadic templates

A **template parameter pack** is a template parameter that accepts zero or more template arguments. This can be a non-type pack, such as `int ... params`, or a type pack, such as `typename ... Types`. It can be preceded by ordinary non-pack parameters. Some examples:

In [7]:
template <typename ... Types>
struct Tuple {} ;

In [8]:
Tuple<> t0;           // Types contains no arguments
Tuple<int> t1;        // Types contains: int
Tuple<int, float> t2; // Types contains: int and float
//Tuple<0> error;     // error: 0 is not a type

In [9]:
template <typename Type, int ... params>
struct Matrix {} ;

In [10]:
Matrix<double,3,4> m ; // Params contains: 3 and 4.

For a variadic function template, on top of a type pack, there is also a **function parameter pack** within the function parameter list, such as `Types ... args`. It can be preceded by ordinary non-pack parameters. For example :

In [11]:
template <typename ... Types>           // Types is the template parameter pack 
void print(char sep, Types ... args) {} // args is the function parameter pack

In [12]:
print(' ');         // args contains no arguments
print(' ', 2, 1.0); // args contains: 2 and 1.0

**Pack expansion** takes place in the body of variadic templates, wherever one use the syntax `<pattern> ...`, where `<pattern>` can be almost any expression which includes one or more parameter packs. For the time being, we will consider only the most simple patterns:
* `Types ...` will expand to a comma-separated list of types.
* `params ...` will expand to a comma-separated list of values.
* `args ...` will expand to a comma-separated list of arguments.

An example below, using a non-type pack:

In [13]:
#include <iostream>
template <int ... params>
void print_ints()
 {
  for ( int i : { params ... } )
    std::cout << i << ' ';
  std::cout << std::endl;
 }

In [14]:
print_ints<1,2,3,4,5>();

1 2 3 4 5 


The previous implementation takes profit that all values of the `Params` pack have the same type. Thus, they can be expanded inside an `std::initializer_list<int>`, which is then used in a range-based for.

## Advanced pack expansion

Instead of simply using a pack name followed by `...`, one can use a **pattern containing a pack name** (or several packs), and the whole pattern will be duplicated into a comma-separated list. The pattern can contain almost anything, including function calls.

In [15]:
int square( int i )
 { return (i*i) ; }

In [16]:
#include <iostream>
template <int ... params>
void print_square_ints()
 {
  for ( int i : { square(params) ... } )
    std::cout << i << ' ';
  std::cout << std::endl;
 }

In [17]:
print_square_ints<1,2,3,4,5>();

1 4 9 16 25 


## Exercise

Try to complete the implementation below of `my_make_tuple`, which is expected to construct an instance of `std::tuple` from any collection of values (replace occurences of /\*\*\*\*\*/):

In [20]:
%%file tmp.perfect-forwarding.cpp

#include <iostream>
#include <tuple>

template< typename ... Types >
auto my_make_tuple( Types ... args )
 { return std::tuple /*****/ ( /*****/ ) ; }

int main()
 {
  auto data = my_make_tuple(3.1416,42) ;
  std::cout << std::get<0>(data) << " " << std::get<1>(data) << std::endl ;
 }

Overwriting tmp.perfect-forwarding.cpp


In [21]:
!rm -f tmp.perfect-forwarding.exe && g++ -std=c++17 tmp.perfect-forwarding.cpp -o tmp.perfect-forwarding.exe

In file included from tmp.perfect-forwarding.cpp:3:
/usr/local/include/c++/10.2.0/tuple: In instantiation of ‘struct std::tuple_element<0, std::tuple<> >’:
/usr/local/include/c++/10.2.0/utility:118:11:   required by substitution of ‘template<long unsigned int __i, class _Tp> using __tuple_element_t = typename std::tuple_element::type [with long unsigned int __i = 0; _Tp = std::tuple<>]’
/usr/local/include/c++/10.2.0/tuple:1315:5:   required by substitution of ‘template<long unsigned int __i, class ... _Elements> constexpr std::__tuple_element_t<__i, std::tuple<_UTypes ...> >&& std::get(const std::tuple<_UTypes ...>&&) [with long unsigned int __i = 0; _Elements = {}]’
tmp.perfect-forwarding.cpp:12:32:   required from here
/usr/local/include/c++/10.2.0/tuple:1277:25: error: static assertion failed: tuple index is in range
 1277 |       static_assert(__i < tuple_size<tuple<>>::value,
      |                     ~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~
tmp.perfect-forwarding.cpp: In function ‘int 

In [ ]:
!./tmp.perfect-forwarding.exe

© *CNRS 2020*  
*Assembled and written by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)